In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd '/content/gdrive/MyDrive/Dataset/image/flowers'
!ls -l

/content/gdrive/MyDrive/Dataset/image/flowers
total 20
drwx------ 2 root root 4096 Dec  6 06:56 daisy
drwx------ 2 root root 4096 Dec  6 06:56 dandelion
drwx------ 2 root root 4096 Dec  6 06:56 rose
drwx------ 2 root root 4096 Dec  6 06:56 sunflower
drwx------ 2 root root 4096 Dec  6 06:56 tulip


In [4]:
'''
Run under google colab with python 3.7.9 and tensorflow 2.3.1
Download data from https://www.kaggle.com/alxmamaev/flowers-recognition
'''
import cv2
import tensorflow as tf
import keras
import os
print(tf.__version__)

2.3.0


In [5]:
data_path=r'/content/gdrive/MyDrive/Dataset/image/flowers'
classes=os.listdir(data_path)
print(len(classes))

5


In [6]:
for i in classes:
    print(i,len(os.listdir('{}/{}'.format(data_path,i))))

rose 784
daisy 769
sunflower 734
tulip 984
dandelion 1055


In [7]:
training_set = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                  labels='inferred',
                                  label_mode='categorical',
                                  image_size = (224, 224),
                                  batch_size = 64,
                                  shuffle=True,
                                  seed=0,
                                  validation_split=0.2,
                                  subset='training')

validation_set = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                  labels='inferred',
                                  label_mode='categorical',
                                  image_size = (224, 224),
                                  batch_size = 64,
                                  shuffle=True,
                                  seed=0,
                                  validation_split=0.2,
                                  subset='validation')


Found 4323 files belonging to 5 classes.
Using 3459 files for training.
Found 4323 files belonging to 5 classes.
Using 864 files for validation.


In [8]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,Flatten,Dense,Input,BatchNormalization,Add

input_shape = (224,224,3)
classes=5

def Conv_block(X,X_shortcut,feature_map,stride,name):
    for i in range(2):
        X = keras.layers.ZeroPadding2D((1, 1),name='padding'+name+str(i+1))(X)
        X = keras.layers.Conv2D(feature_map, (3, 3), strides=stride, name='Conv'+name+str(i+1))(X) 
        X = keras.layers.BatchNormalization(axis=3, name='bn_conv'+name+str(i+1))(X)
        if i == 0:
            X = keras.layers.Activation('relu',name='RELU_'+name+str(i+1))(X)
        else:
            X=keras.layers.Add(name='Skip_Connect_'+name)([X, X_shortcut])
            X = keras.layers.Activation('relu',name='RELU_'+name)(X)
            X_shortcut=X
    return X,X_shortcut

def skip_connect_block(X,X_shortcut,feature_map,name):
    X_shortcut=keras.layers.ZeroPadding2D((1, 1),name='skip_connect_padding'+name)(X_shortcut)
    X_shortcut = keras.layers.Conv2D(feature_map, (3, 3), strides=(2,2), name='skip_connect_Conv'+name)(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3, name='skip_connect_bn'+name)(X_shortcut)
    
    for i in range(2):
        X = keras.layers.ZeroPadding2D((1, 1),name='padding'+name+str(i+1))(X)
        if i==0:
            X = keras.layers.Conv2D(feature_map, (3, 3), strides=(2,2), name='Conv'+name+str(i+1))(X) 
        else:
            X = keras.layers.Conv2D(feature_map, (3, 3), strides=(1,1), name='Conv'+name+str(i+1))(X) 
            
        X = keras.layers.BatchNormalization(axis=3, name='bn_conv'+name+str(i+1))(X)
        if i == 0:
            X = keras.layers.Activation('relu',name='RELU_'+name+str(i+1))(X)
        else:
            X=keras.layers.Add(name='Skip_Connect_'+name)([X, X_shortcut])
            X = keras.layers.Activation('relu',name='RELU_'+name)(X)
            X_shortcut=X
    return X,X_shortcut
#Model
def Resnet34(input_shape,classes):
    X_input = keras.layers.Input(input_shape,name='Input')
    '''
    Stardardization of image
    '''
    X = keras.layers.experimental.preprocessing.Rescaling(1./255)(X_input)
    X = keras.layers.ZeroPadding2D((3, 3),name='padding_0')(X)
    
    #Conv1
    X = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), name='Conv1')(X)
    X = keras.layers.BatchNormalization(axis=3, name='bn_conv1')(X)
    X = keras.layers.ZeroPadding2D((1, 1),name='padding_1')(X)
    X = keras.layers.Activation('relu',name='RELU_1')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), name='max2d_1')(X)
    
    X_shortcut=X
    
    #Conv2
    X,X_shortcut=Conv_block(X,X_shortcut,64,(1, 1),'2.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,64,(1, 1),'2.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,64,(1, 1),'2.3.')
    
    #Conv3
    X,X_shortcut=skip_connect_block(X,X_shortcut,128,'3.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,128,(1, 1),'3.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,128,(1, 1),'3.3.')
    X,X_shortcut=Conv_block(X,X_shortcut,128,(1, 1),'3.4.')
    
    #Conv4
    X,X_shortcut=skip_connect_block(X,X_shortcut,256,'4.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.3.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.4.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.5.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.6.')
    
    #Conv5
    X,X_shortcut=skip_connect_block(X,X_shortcut,512,'5.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,512,(1, 1),'5.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,512,(1, 1),'5.3.')
    
    X = keras.layers.AveragePooling2D((2,2), name="avg_pool")(X)
    #Fully connect
    X = keras.layers.Flatten(name='Flatten')(X)
    X=keras.layers.Dense(classes, activation='softmax', name='Fully_connected')(X)
    model = keras.models.Model(inputs = X_input, outputs = X, name='ResNet')
    return model

model=Resnet34(input_shape,classes)
print(model.summary())

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           Input[0][0]                      
__________________________________________________________________________________________________
padding_0 (ZeroPadding2D)       (None, 230, 230, 3)  0           rescaling[0][0]                  
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 64) 9472        padding_0[0][0]                  
_____________________________________________________________________________________________

In [9]:
#earlystop=tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=3)
loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])
history=model.fit(training_set,epochs=50)

Epoch 1/50
55/55 [==============================] - 1023s 19s/step - loss: 2.7297 - accuracy: 0.2897
Epoch 2/50
55/55 [==============================] - 28s 502ms/step - loss: 1.6073 - accuracy: 0.3727
Epoch 3/50
55/55 [==============================] - 28s 507ms/step - loss: 1.3392 - accuracy: 0.4993
Epoch 4/50
55/55 [==============================] - 28s 511ms/step - loss: 1.3403 - accuracy: 0.5447
Epoch 5/50
55/55 [==============================] - 28s 517ms/step - loss: 1.2754 - accuracy: 0.5635
Epoch 6/50
55/55 [==============================] - 29s 519ms/step - loss: 1.2160 - accuracy: 0.5704
Epoch 7/50
55/55 [==============================] - 29s 519ms/step - loss: 1.1279 - accuracy: 0.6094
Epoch 8/50
55/55 [==============================] - 29s 521ms/step - loss: 1.1299 - accuracy: 0.6088
Epoch 9/50
55/55 [==============================] - 29s 522ms/step - loss: 1.1208 - accuracy: 0.6265
Epoch 10/50
55/55 [==============================] - 29s 525ms/step - loss: 1.0993 - accura